# Extracting events from TCRM database

This notebook provides an interactive way of interrogating the event database generated by TCRM. You can select a location, then extract all events that pass within a defined distance of the location. 

In [1]:
%matplotlib notebook
from __future__ import print_function, division

In [5]:
import os
import io
import sys

import database
import numpy as np
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap

import matplotlib.cm as cm
import pandas as pd


from Utilities.config import ConfigParser
from Utilities.track import ncReadTrackData


# Import widgets for interactive notebook
from ipywidgets import interact, fixed, Dropdown, FloatSlider, interact_manual
import ipywidgets as widgets

import seaborn as sns
sns.set_context("poster")
sns.set_style("whitegrid")

In [6]:
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import shapely.geometry as sg


In [7]:
configstr = """
[DataProcess]
InputFile=/home/tcrm/data/Allstorms.ibtracs_wmo.v03r09.csv
Source=IBTRACS
StartSeason=1981
FilterSeasons=False

[Region]
; Domain for windfield and hazard calculation
gridLimit={'xMin':175.,'xMax':185.,'yMin':-23.0,'yMax':-13.0}
gridSpace={'x':1.0,'y':1.0}
gridInc={'x':1.0,'y':0.5}

[TrackGenerator]
NumSimulations=10000
YearsPerSimulation=1
SeasonSeed=68876543
TrackSeed=334825
TimeStep=1.0

[Input]
landmask = C:/WorkSpace/tcrm/input/landmask.nc
mslpfile = C:/WorkSpace/tcrm/MSLP/slp.day.ltm.nc
datasets = IBTRACS,LTMSLP

[Output]
Path=C:/workSpace/data/share/tcrm/fiji

[Hazard]
Years=2,5,10,20,25,50,100,200,250,500,1000,2000,2500,5000
MinimumRecords=10
CalculateCI=False

[Logging]
LogFile=C:/WorkSpace/data/share/tcrm/fiji/log/fiji.log
LogLevel=INFO
Verbose=False

[IBTRACS]
; Input data file settings
url = ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r06/wmo/csv/Allstorms.ibtracs_wmo.v03r09.csv.gz
path = C:/WorkSpace/tcrm/input/
filename = Allstorms.ibtracs_wmo.v03r09.csv
columns = tcserialno,season,num,skip,skip,skip,date,skip,lat,lon,skip,pressure
fielddelimiter = ,
numberofheadinglines = 3
pressureunits = hPa
lengthunits = km
dateformat = %Y-%m-%d %H:%M:%S
speedunits = kph

[LTMSLP]
; MSLP climatology file settings
URL = ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/surface/slp.day.1981-2010.ltm.nc
path = C:/WorkSpace/tcrm/MSLP
filename = slp.day.ltm.nc

[Process]
DatFile=C:/WorkSpace/data/share/tcrm/fiji/process/dat/fiji.dat
ExcludePastProcessed=False
"""

In [12]:
working_dir = '/g/data/w85/cek156/db10kyr/'
#working_dir = 'C:/Users/u37443/TCRM/output/10kyr_CA/'
configFile = working_dir + 'tcrm2.0.ini'
config = ConfigParser()
config.read(configFile)

db = database.HazardDatabase(configstr)
locations = db.getLocations()
locNameList = list(locations['locName'])
outputPath = config.get('Output', 'Path')

OperationalError: unable to open database file

In [10]:
def loadTrack(trackId):
    trackNum, trackYear = int(trackId.split('-')[0]), int(trackId.split('-')[1])
    trackFile = os.path.join(outputPath, 'tracks', 'tracks.{0:05d}.nc'.format(trackYear))
    tracks = ncReadTrackData(trackFile)
    
    return [t for t in tracks if t.trackId==(trackNum, trackYear)][0]

def getTracks(recs):
    tracks = []
    for rec in recs:
        trackId = rec['eventId']
        track = loadTrack(trackId)
        tracks.append(track)
    return tracks

In [11]:
def plotTracks(locName, distance=50.):
    locId = locations['locId'][locations['locName']==locName][0]
    locLon = locations['locLon'][locations['locId']==locId][0]
    locLat = locations['locLat'][locations['locId']==locId][0]
    recs = database.locationPassageWindSpeed(db, locId, 40, distance)
    tracks = getTracks(recs)
    
    domain = config.geteval('Region', 'gridLimit')
    fig = plt.figure()
    
    prj = ccrs.PlateCarree(central_longitude=180)
    prj0 = ccrs.PlateCarree(central_longitude=0)
    ax = plt.axes(projection=prj)
    ax.coastlines(resolution='50m', color='black', linewidth=1)
    for t in tracks:
        plt.plot(t.Longitude, t.Latitude, color='gray', linewidth=1, transform=prj0, alpha=0.25)
        
    gl = ax.gridlines(crs=prj0, draw_labels=True,
                      linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = False
    gl.xlocator = mticker.FixedLocator(range(120, 210, 15))
    gl.ylocator = mticker.FixedLocator(range(-30, 0, 5))

    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 10, 'color': 'black'}
    gl.ylabel_style = {'size': 10, 'color': 'black'}

    ax.set_title("TCs passing within {0} km of {1}".format(int(distance), locName))
    fig.show()
    #xx = np.arange(domain['xMin'], domain['xMax'] + 0.1, 0.1)
    #yy = np.arange(domain['yMin'], domain['yMax'] + 0.1, 0.1)
    #[xgrid, ygrid] = np.meshgrid(xx,yy)
    #fig, axes = plt.subplots()
    #mapkwargs = dict(llcrnrlon=domain['xMin']-10,
    #                 llcrnrlat=domain['yMin'],
    #                 urcrnrlon=domain['xMax'],
    #                 urcrnrlat=domain['yMax'],
    #                 resolution='f',
    #                 projection='merc')
    #mapobj = Basemap(ax=axes, **mapkwargs)
    #mx, my = mapobj(xgrid, ygrid)
    #xmin = mapobj.llcrnrlon
    #xmax = mapobj.urcrnrlon
    #ymin = mapobj.llcrnrlat
    #ymax = mapobj.urcrnrlat

    #dx = abs(xmin - xmax)
    #dy = abs(ymin - ymax)
    #dd = max(dx, dy)
    #gr_opts = np.array([30., 10., 5., 4., 2.])
    #min_gr = 5
    #try:
    #    dl = gr_opts[np.where((dd/gr_opts) >= min_gr)[0][0]]
    #except IndexError:
    #    dl = 2.

    #meridians = np.arange(dl*np.floor(xmin / dl),
    #                      dl*np.ceil(xmax / dl) + dl, dl)
    #parallels = np.arange(dl*np.floor(ymin / dl),
    #                      dl*np.ceil(ymax / dl) + dl, dl)

    #mapobj.drawparallels(parallels, linewidth=0.25,
    #                     labels=[1, 0, 0, 1], style="italic")
    #mapobj.drawmeridians(meridians, linewidth=0.25,
    #                     labels=[1, 0, 0, 1], style='italic')

    #mapobj.fillcontinents(color="#F4B342",
    #                      lake_color="#BEE8FF",
    #                      zorder=0)
    #mapobj.drawcoastlines(linewidth=.5, color="k")
    #mapobj.drawmapboundary(fill_color="#BEE8FF")
    #for track in tracks:
    #    mlon, mlat = mapobj(track.Longitude, track.Latitude)
    #    mlocLon, mlocLat = mapobj(locLon, locLat)
    #    mapobj.plot(mlon, mlat, linewidth=1)
    #    mapobj.plot(mlocLon, mlocLat, marker='o', color='r', markersize=5)

    #axes.set_title("TCs passing within {0} km of {1}".format(int(distance), locName))
    #fig.tight_layout()
    #plt.show()

In [ ]:
interact_manual(plotTracks, 
                locName=Dropdown(options=locNameList, description="Location"),
                distance=FloatSlider(value=50, min=5, max=100, step=1.0, 
                                     description="Closest passage", 
                                     continuous_update=False)
               )